# Grape diseases and pests classification

**Chi Tran - CMPE 295**

## Data structure

I will apply an unsupervised machine learning model to recognize and classify the images. Therefore, Keras ImageDataGenerator is the best choice. As mentioned on the website: https://medium.com/@vijayabhaskar96/tutorial-image-classification-with-keras-flow-from-directory-and-generators-95f75ebe5720, my data will be divided into sub-folders that corresponding to image classes under train, validation, and test folders.
![](https://cdn-images-1.medium.com/max/800/1*HpvpA9pBJXKxaPCl5tKnLg.jpeg)


In [1]:
from __future__ import (absolute_import, 
                    division, 
                    print_function, 
                    unicode_literals)
import os
import glob
import time
import numpy as np
import pandas as pd
import math
import seaborn as sns
import random
import shutil
import pylab
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects
import tensorflow as tf
import h5py
import keras
from keras import models, layers, optimizers 
from keras.models import load_model
from keras import backend as K
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from mpl_toolkits.axes_grid1 import ImageGrid
from keras.applications import imagenet_utils
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.manifold import TSNE
from ipywidgets import interact, interactive, fixed, interact_manual, Layout, Button, Box, VBox, HBox, FloatText, Textarea, Dropdown, Label, IntSlider
import ipywidgets as widgets

# Set random seed for consistency
random.seed(150)
np.random.seed(150)
tf.random.set_seed(150)

# Displays all outputs of a cell 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

%matplotlib inline
plt.rcParams['figure.figsize'] = [15, 4]

print('TensorFlow version:', tf.__version__)
print('Keras version:', tf.keras.__version__)
    


Using TensorFlow backend.


TensorFlow version: 2.2.0
Keras version: 2.3.0-tf


### Create data folders

In [2]:
import importlib
from setup import data_structure as ds, data_normalization as dn
importlib.reload(ds)
importlib.reload(dn)

<module 'setup.data_structure' from 'C:\\Users\\chitt\\Chi Tran_CMPE295\\setup\\data_structure.py'>

<module 'setup.data_normalization' from 'C:\\Users\\chitt\\Chi Tran_CMPE295\\setup\\data_normalization.py'>

In [3]:
ds.create_data_folders()

### Collect Images in each class

In [4]:
ds.collect_images()

There are 200 images in phylloxera class
There are 534 images in mealybug class
There are 120 images in black measles class
There are 120 images in black rot class
There are 156 images in leaf blight class
There are 120 images in healthy class


### Show data distribution

### Checking data allocation after spliting

In [5]:
# Checking data
total_train_phylloxera = len(os.listdir(ds.train_phylloxera))
total_train_mealybug = len(os.listdir(ds.train_mealybug))
total_train_b_measles = len(os.listdir(ds.train_b_measles))
total_train_b_rot = len(os.listdir(ds.train_b_rot))
total_train_leaf_blight = len(os.listdir(ds.train_leaf_blight))
total_train_healthy = len(os.listdir(ds.train_healthy))

total_valid_phylloxera = len(os.listdir(ds.val_phylloxera))
total_valid_mealybug = len(os.listdir(ds.val_mealybug))
total_valid_b_measles = len(os.listdir(ds.val_b_measles))
total_valid_b_rot = len(os.listdir(ds.val_b_rot))
total_valid_leaf_blight = len(os.listdir(ds.val_leaf_blight))
total_valid_healthy = len(os.listdir(ds.val_healthy))

total_test_phylloxera = len(os.listdir(ds.test_phylloxera))
total_test_mealybug = len(os.listdir(ds.test_mealybug))
total_test_b_measles = len(os.listdir(ds.test_b_measles))
total_test_b_rot = len(os.listdir(ds.test_b_rot))
total_test_leaf_blight = len(os.listdir(ds.test_leaf_blight))
total_test_healthy = len(os.listdir(ds.test_healthy))

total_train = total_train_b_measles + total_train_b_rot + \
                total_train_leaf_blight + total_train_healthy + total_train_phylloxera + total_train_mealybug
total_valid = total_valid_b_measles + total_valid_b_rot + \
              total_valid_leaf_blight + total_valid_healthy + total_valid_phylloxera + total_valid_mealybug
total_test = total_test_b_measles + total_test_b_rot + \
              total_test_leaf_blight + total_test_healthy + total_test_phylloxera + total_test_mealybug

def check_dist(total, phylloxera, mealybug, black_measles, black_rot, leaf_blight, healthy):
  print('Phylloxera class occupies: {:.2f}%'.format((phylloxera / total)*100.0))
  print('Mealybug class occupies: {:.2f}%'.format((mealybug / total)*100.0))
  print('Black measles class occupies: {:.2f}%'.format((black_measles/total)*100.0))
  print('Black rot class occupies: {:.2f}%'.format((black_rot / total)*100))
  print('Left blight class occupies: {:.2f}%'.format((leaf_blight / total)*100.0))
  print('Healthy class occupies: {:.2f}%'.format((healthy / total)*100.0))
  print()
  
print('Total images:', total_train + total_valid + total_test)
print('Total phylloxera:', total_train_phylloxera + total_valid_phylloxera + total_test_phylloxera)
print('Total mealybug:', total_train_mealybug + total_valid_mealybug + total_test_mealybug)
print('Total black measles:', total_train_b_measles + total_valid_b_measles + total_test_b_measles)
print('Total black rot:', total_train_b_rot + total_valid_b_rot + total_test_b_rot)
print('Total leaf blight:', total_train_leaf_blight + total_valid_leaf_blight + total_test_leaf_blight)
print('Total healthy:', total_train_healthy + total_valid_healthy + total_test_healthy)
print()

print('Total of training phylloxera images:', total_train_phylloxera)
print('Total of training mealybug images:', total_train_mealybug)
print('Total of training black measles images:', total_train_b_measles)
print('Total of training black rot images:', total_train_b_rot)
print('Total of training leaf blight images:', total_train_leaf_blight)
print('Total of training healthy images:', total_train_healthy)
print()

print('Total of validation phylloxera images:', total_valid_phylloxera)
print('Total of validation mealybug images:', total_valid_mealybug)
print('Total of validation black measles images:', total_valid_b_measles)
print('Total of validation black rot images:', total_valid_b_rot)
print('Total of validation leaf blight images:', total_valid_leaf_blight)
print('Total of validation healthy images:', total_valid_healthy)
print()

print('Total of test phylloxera images:', total_test_phylloxera)
print('Total of test mealybug images:', total_test_mealybug)
print('Total of test black measles images:', total_test_b_measles)
print('Total of test black rot images:', total_test_b_rot)
print('Total of test leaf blight images:', total_test_leaf_blight)
print('Total of test healthy images:', total_test_healthy)
print()

print('Training distribution')
check_dist(total_train, total_train_b_measles, total_train_b_rot,
                    total_train_leaf_blight, total_train_healthy, total_train_phylloxera, total_train_mealybug)
print('Validation distribution')
check_dist(total_valid, total_valid_b_measles, total_valid_b_rot,
                    total_valid_leaf_blight, total_valid_healthy, total_valid_phylloxera, total_valid_mealybug)
print('Test distribution')
check_dist(total_test, total_test_b_measles, total_test_b_rot,
                    total_test_leaf_blight, total_test_healthy, total_test_phylloxera, total_test_mealybug)


Total images: 1250
Total phylloxera: 200
Total mealybug: 534
Total black measles: 120
Total black rot: 120
Total leaf blight: 156
Total healthy: 120

Total of training phylloxera images: 128
Total of training mealybug images: 342
Total of training black measles images: 77
Total of training black rot images: 77
Total of training leaf blight images: 100
Total of training healthy images: 77

Total of validation phylloxera images: 32
Total of validation mealybug images: 85
Total of validation black measles images: 19
Total of validation black rot images: 19
Total of validation leaf blight images: 24
Total of validation healthy images: 19

Total of test phylloxera images: 40
Total of test mealybug images: 107
Total of test black measles images: 24
Total of test black rot images: 24
Total of test leaf blight images: 32
Total of test healthy images: 24

Training distribution
Phylloxera class occupies: 9.61%
Mealybug class occupies: 9.61%
Black measles class occupies: 12.48%
Black rot class oc

## Data Analysis

### Observe Data Images

In [6]:
from utilities import data_observation as do
importlib.reload(do)

<module 'utilities.data_observation' from 'C:\\Users\\chitt\\Chi Tran_CMPE295\\utilities\\data_observation.py'>

### Load images into numpy arrays

In [7]:
# Load numpy arrays from the folder
phylloxera_numpy = np.load(os.path.join(ds.bin_data, 'phylloxera.npy'))
mealybug_numpy = np.load(os.path.join(ds.bin_data, 'mealybug.npy'))
b_measles_numpy = np.load(os.path.join(ds.bin_data, 'black_measles.npy'))
b_rot_numpy = np.load(os.path.join(ds.bin_data, 'black_rot.npy'))
leaf_blight_numpy = np.load(os.path.join(ds.bin_data, 'leaf_blight.npy'))
healthy_numpy = np.load(os.path.join(ds.bin_data, 'healthy.npy'))

In [8]:
# Reshape numpy arrays into sizes of 256x256x3 as input data
phylloxera_numpy = phylloxera_numpy.reshape(128, 256*256*3)
mealybug_numpy = mealybug_numpy.reshape(342, 256*256*3)
b_measles_numpy = b_measles_numpy.reshape(77, 256*256*3)
b_rot_numpy = b_rot_numpy.reshape(77, 256*256*3)
leaf_blight_numpy = leaf_blight_numpy.reshape(100, 256*256*3)
healthy_numpy = healthy_numpy.reshape(77, 256*256*3)

### Visualize data

In [9]:
from utilities import data_visualization as dv
importlib.reload(dv)

<module 'utilities.data_visualization' from 'C:\\Users\\chitt\\Chi Tran_CMPE295\\utilities\\data_visualization.py'>

In [10]:
# Create data labels for all classes in 1D
phylloxera_label = np.full(phylloxera_numpy.shape[0], 'phylloxera')
mealybug_label = np.full(mealybug_numpy.shape[0], 'mealybug')
b_measles_label = np.full(b_measles_numpy.shape[0], 'black_measles')
b_rot_label = np.full(b_rot_numpy.shape[0], 'black_rot')
leaf_blight_label = np.full(leaf_blight_numpy.shape[0], 'leaf_blight')
healthy_label = np.full(healthy_numpy.shape[0], 'healthy')

labels = (phylloxera_label, mealybug_label, b_measles_label, b_rot_label, leaf_blight_label, healthy_label)
labels = np.hstack(labels) 

In [11]:
# Create a stack of numpy arrays 
tuple_data = (phylloxera_numpy, mealybug_numpy, b_measles_numpy, b_rot_numpy, leaf_blight_numpy, healthy_numpy)

# normalize tuple data
nomalized_data = np.vstack(tup=tuple_data)/255. 

In [12]:
# Run PCA on nomalized data
start_time = time.time()
pca = PCA(n_components=60)
nomalized_data_pca = pca.fit_transform(nomalized_data)
end_time = time.time()

print('This process took {:.2f} seconds.'.format(end_time-start_time))

This process took 9.48 seconds.


In [13]:
# Run t-SNE on data
start_time = time.time()
tsne = TSNE(n_components=2)
nomalized_data_tsne = tsne.fit_transform(nomalized_data_pca)
end_time = time.time()

print('This process took {:.2f} seconds'.format(end_time-start_time))

This process took 5.75 seconds


## Training data

### Libraries

### Set up hyper-parameters

In [14]:
image_height, image_width = 256, 256
batch_size = 30
epochs = 3

print('Total number of training data:', total_train)
print('Total number of validation data:', total_valid)
print('Total number of test data:', total_test)

Total number of training data: 801
Total number of validation data: 198
Total number of test data: 251


### Training from Scratch

In [15]:
# Rescale data
training_datagenerator = ImageDataGenerator(rescale=1./255)
test_datagenerator = ImageDataGenerator(rescale=1./255)

training_gen = training_datagenerator.flow_from_directory(
                    ds.train_fold,
                    target_size=(image_height, image_width),
                    color_mode='rgb',
                    batch_size=batch_size,
                    class_mode='categorical',
                    shuffle=True, 
                    seed=30)

valid_gen = test_datagenerator.flow_from_directory(
                          ds.val_fold,
                          target_size=(image_height, image_width),
                          color_mode='rgb',
                          batch_size=batch_size,
                          class_mode='categorical')

Found 801 images belonging to 6 classes.
Found 198 images belonging to 6 classes.


In [16]:
# Create a new validation generator for evaluation
valid_test_gen = test_datagenerator.flow_from_directory(
                ds.val_fold,
                target_size=(image_height, image_width),
                color_mode='rgb',
                batch_size=1,
                class_mode='categorical',
                shuffle=False)

Found 198 images belonging to 6 classes.


#### Build custom metrics 

In [17]:
from libraries import custom_metrics as cm, create_diagram as cd, multi_class_TF as mc, early_stopping as es, time_hist as th

In [18]:

from libraries import performance_visualization as pv, neptune_callback as nc
importlib.reload(cm)

importlib.reload(mc)
importlib.reload(es)
importlib.reload(th)
importlib.reload(pv)
importlib.reload(nc)

NVMLError: NVML Shared Library Not Found - GPU usage metrics may not be reported.


https://ui.neptune.ai/chittp/sandbox/e/SAN-103


<module 'libraries.custom_metrics' from 'C:\\Users\\chitt\\Chi Tran_CMPE295\\libraries\\custom_metrics.py'>

<module 'libraries.multi_class_TF' from 'C:\\Users\\chitt\\Chi Tran_CMPE295\\libraries\\multi_class_TF.py'>

<module 'libraries.early_stopping' from 'C:\\Users\\chitt\\Chi Tran_CMPE295\\libraries\\early_stopping.py'>

<module 'libraries.time_hist' from 'C:\\Users\\chitt\\Chi Tran_CMPE295\\libraries\\time_hist.py'>

<module 'libraries.performance_visualization' from 'C:\\Users\\chitt\\Chi Tran_CMPE295\\libraries\\performance_visualization.py'>

https://ui.neptune.ai/chittp/sandbox/e/SAN-104


<module 'libraries.neptune_callback' from 'C:\\Users\\chitt\\Chi Tran_CMPE295\\libraries\\neptune_callback.py'>

In [19]:
importlib.reload(cd)

<module 'libraries.create_diagram' from 'C:\\Users\\chitt\\Chi Tran_CMPE295\\libraries\\create_diagram.py'>

In [20]:
def recall(y_true, y_pred):
    y_true = K.ones_like(y_true) 
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    all_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    
    recall = true_positives / (all_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    y_true = K.ones_like(y_true) 
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_score(y_true, y_pred):
    precision1 = precision(y_true, y_pred)
    recall1 = recall(y_true, y_pred)
    return 2*((precision1*recall1)/(precision1+recall1+K.epsilon()))

#### Define custom callback for prediction

In [21]:
early_stopping = es.EarlyStoppingAtMinLoss()

#### Callback class for time history

In [22]:
time_callback = th.TimeHistory()

#### Multi class true-positive

In [23]:
multiTF = mc.MulticlassTruePositives()

In [24]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual, Layout, Button, Box, VBox, HBox, FloatText, Textarea, Dropdown, Label, IntSlider
import ipywidgets as widgets
from bqplot import *
import bqplot.pyplot as plt

In [25]:
widgets.IntSlider(min=-10, max=30, step=1, value=10)

IntSlider(value=10, max=30, min=-10)

In [26]:

dropdown = widgets.Dropdown(description='Embedding', options=['Build Dataset', 'Collect Data'])
dropdown

Dropdown(description='Embedding', options=('Build Dataset', 'Collect Data'), value='Build Dataset')

In [27]:
if dropdown.value=="Collect Data":
    print('Chi')
    import cv2
    import glob
    import matplotlib.pyplot as plt
    import PIL

    images = [cv2.imread(file) for file in glob.glob('output/*.PNG')]
    for im in images: 
        fig = plt.figure(figsize=(8,7))
        plt.imshow(im)
        #plt.show()
    

In [28]:
play = widgets.Play(
    value=50,
    min=0,
    max=100,
    step=1,
    interval=500,
    description="Press play",
    disabled=False
)
slider = widgets.IntSlider()
widgets.jslink((play, 'value'), (slider, 'value'))
widgets.HBox([play, slider])

Link(source=(Play(value=50, description='Press play', interval=500), 'value'), target=(IntSlider(value=0), 'va…

In [29]:
def renderWidget(width=None, height=None):
    """ Render a pygal chart into a Jupyter Notebook """
    from ipywidgets import HTML

   
    html = '<!DOCTYPE html><html><head><style>body {background-image: url("img_data/black_rot/black_rot1.jpg");}</style></head><body></body></html>'

   

    return HTML(html) 

In [30]:
di=renderWidget(300, 400)

In [31]:
di

HTML(value='<!DOCTYPE html><html><head><style>body {background-image: url("img_data/black_rot/black_rot1.jpg")…

In [32]:
color_buttons = widgets.ToggleButtons(
    options=['blue', 'red', 'green'],
    description='Color:',
)
color_buttons

ToggleButtons(description='Color:', options=('blue', 'red', 'green'), value='blue')

In [33]:
title_textbox = widgets.Text(
    value='Hello World',
    description='Title:',
)
title_textbox

Text(value='Hello World', description='Title:')

In [34]:
color_picker = widgets.ColorPicker(
    concise=True,
    description='Background color:',
    value='#efefef',
)
color_picker

ColorPicker(value='#efefef', concise=True, description='Background color:')

In [35]:
grid_button = widgets.ToggleButton(
    value=False,
    description='Grid',
    icon='check'
)
grid_button

ToggleButton(value=False, description='Grid', icon='check')

In [36]:
button = widgets.Button(
    description='Plot',
)
button

Button(description='Plot', style=ButtonStyle())

In [37]:
#===========================================Buttons for tab Data Management===================================================#
btn_folder = widgets.Button(
    description='Build Dataset',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Create data folders for all classes.'
    #icon='check' # (FontAwesome names without the `fa-` prefix)
)
btn_collection = widgets.Button(
    description='Collect Data',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Count total images in each class.',
    color = 'green'
    #icon='check' # (FontAwesome names without the `fa-` prefix)
)

btn_rename_img = widgets.Button(
    description='Rename Images',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Rename all images follow a naming convention.'
    #icon='check' # (FontAwesome names without the `fa-` prefix)
)
btn_split_data = widgets.Button(
    description='Split Data',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Split data into train, test, and validation folders.',
    color = 'green'
    #icon='check' # (FontAwesome names without the `fa-` prefix)
)

#=============================================Buttons for tab Data Analysis===================================================#

btn_observation = widgets.Button(
    description='Data Observation',
    disabled=False,
    button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='View sample data in all classes.',
    color = 'blue'
    #icon='check' # (FontAwesome names without the `fa-` prefix)
)
btn_distribution = widgets.Button(
    description='Data Distribution',
    disabled=False,
    button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Display data distribution in each class.',
    color = 'blue'
    #icon='check' # (FontAwesome names without the `fa-` prefix)
)

btn_visualization = widgets.Button(
    description='Data Visualization',
    disabled=False,
    button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Visualize data in all classes.',
    color = 'blue'
    #icon='check' # (FontAwesome names without the `fa-` prefix)
)

#=============================================Buttons for tab Quality Tracking===============================================#

btn_acc_loss = widgets.Button(
    description='View Acc/Loss',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='View accuracy and loss of training and validation.'
    #icon='check' # (FontAwesome names without the `fa-` prefix)
)
btn_confusion_matrix = widgets.Button(
    description='Confusion Matrix',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='View confusion matrix.'
    #icon='check' # (FontAwesome names without the `fa-` prefix)
)

btn_epoch_cf = widgets.Button(
    description='Confusion matrix/epoch',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Display confusion matrix for each epoch.',
    color = 'green'
    #icon='check' # (FontAwesome names without the `fa-` prefix)
)

btn_roc = widgets.Button(
    description='Show ROC',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Display ROC for each epoch.',
    color = 'green'
    #icon='check' # (FontAwesome names without the `fa-` prefix)
)

btn_mis_images = widgets.Button(
    description='Misclassified Images',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Get all misclassified images.'
    #icon='check' # (FontAwesome names without the `fa-` prefix)
)
btn_hidden_layers = widgets.Button(
    description='Hidden Layers',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='View all hidden layers.',
    color = 'green'
    #icon='check' # (FontAwesome names without the `fa-` prefix)
)

btn_layer_info = widgets.Button(
    description='Layers Information',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='View layer information.'
    #icon='check' # (FontAwesome names without the `fa-` prefix)
)
btn_activations = widgets.Button(
    description='Display Activations',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Display activations of each image.',
    color = 'green'
    #icon='check' # (FontAwesome names without the `fa-` prefix)
)

btn_heatmaps = widgets.Button(
    description='Display Heatmaps',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Display heatmaps of each image.'
    #icon='check' # (FontAwesome names without the `fa-` prefix)
)
btn_weights = widgets.Button(
    description='Weights/Bias',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Display weights and bias on all layers.',
    color = 'green'
    #icon='check' # (FontAwesome names without the `fa-` prefix)
)

btn_inbound = widgets.Button(
    description='Inbound Nodes',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Display all inbound nodes.'
    #icon='check' # (FontAwesome names without the `fa-` prefix)
)
btn_outbound = widgets.Button(
    description='Outbound Nodes',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Display all outbound nodes.',
    color = 'green'
    #icon='check' # (FontAwesome names without the `fa-` prefix)
)

#=============================================Images for tab Quality Tracking========================
accFile = open("images/acc-loss.GIF", "rb")
acc = accFile.read()
acc1 = widgets.Image(
    value=acc,
    format='png',
    width=256,
    height=256,
)

confFile = open("images/confusion-matrix.GIF", "rb")
conf = confFile.read()
conf1 = widgets.Image(
    value=conf,
    format='png',
    width=256,
    height=256,
)

confBFile = open("images/confusion-matrix1.GIF", "rb")
confB = confBFile.read()
confB1 = widgets.Image(
    value=confB,
    format='png',
    width=256,
    height=256,
)

rocFile = open("images/roc.GIF", "rb")
roc = rocFile.read()
roc1 = widgets.Image(
    value=roc,
    format='png',
    width=256,
    height=256,
)

misFile = open("images/missed1.JPG", "rb")
missed = misFile.read()
missed1 = widgets.Image(
    value=missed,
    format='png',
    width=120,
    height=120,
)

hlayerFile = open("images/hidden.GIF", "rb")
hlayer = hlayerFile.read()
hlayer1 = widgets.Image(
    value=hlayer,
    format='png',
    width=256,
    height=256,
)

layerFile = open("images/layer.GIF", "rb")
layer = layerFile.read()
layer1 = widgets.Image(
    value=layer,
    format='png',
    width=256,
    height=256,
)

actFile = open("images/activation.GIF", "rb")
act = actFile.read()
act1 = widgets.Image(
    value=act,
    format='png',
    width=256,
    height=256,
)

heatFile = open("images/heatmap.GIF", "rb")
heatmaps = heatFile.read()
heatmaps1 = widgets.Image(
    value=heatmaps,
    format='png',
    width=256,
    height=256,
)

weightFile = open("images/weights.GIF", "rb")
weights = weightFile.read()
weights1 = widgets.Image(
    value=weights,
    format='png',
    width=256,
    height=256,
)

inFile = open("images/innode.GIF", "rb")
inI = inFile.read()
in1 = widgets.Image(
    value=inI,
    format='png',
    width=256,
    height=256,
)

outFile = open("images/outnode.GIF", "rb")
out = outFile.read()
out1 = widgets.Image(
    value=out,
    format='png',
    width=256,
    height=256,
)

#=============================================Text labels for tab Quality Tracking========================
accLabel = widgets.HTML("<width='200px'><h8 class='text-left'><font color='green'>View accuracy and loss of all epoches for trainingand validation datasets.</font></h8></width>")
confLabel = widgets.HTML("<width='200px'><h8 class='text-left'><font color='green'>Display confusion matrix and classification report of the training model on all classes.</font></h8></width>")
confELabel = widgets.HTML("<width='200px'><h8 class='text-left'><font color='green'>This function shows users how the model is improved after each epoch.</font></h8></width>")
rocLabel = widgets.HTML("<width='200px'><h8 class='text-left'><font color='green'>Show receiver operating characteristic (ROC) curve on each epoch.</font></h8></width>")
misLabel = widgets.HTML("<width='200px'><h8 class='text-left'><font color='green'>Collect all misclassified images & display with the name of images.</font></h8></width>")
hiddenLabel = widgets.HTML("<width='200px'><h8 class='text-left'><font color='green'>Display all hidden layers in the model.</font></h8></width>")
layerLabel = widgets.HTML("<width='200px'><h8 class='text-left'><font color='green'>Extract detailed information of each layer of the training model.</font></h8></width>")
actLabel = widgets.HTML("<width='200px'><h8 class='text-left'><font color='green'>Present activations of an selected image on each training model's layer.</font></h8></width>")
heatLabel = widgets.HTML("<width='200px'><h8 class='text-left'><font color='green'>Show heatmaps of an selected image on each training model's layer.</font></h8></width>")
weightLabel = widgets.HTML("<width='200px'><h8 class='text-left'><font color='green'>Display wieghts and bias on each training model's layer.</font></h8></width>")
inLabel = widgets.HTML("<width='200px'><h8 class='text-left'><font color='green'>Extract all inbound nodes on each layer.</font></h8></width>")
outLabel = widgets.HTML("<width='200px'><h8 class='text-left'><font color='green'>Extract all outbound nodes on each layer.</font></h8></width>")

#======================================Buttons for tab AI Model Tracking===========================================================#

#========================================Buttons for tab About===========================================================#
teamFile = open("images/team.PNG", "rb")
team = teamFile.read()
team1 = widgets.Image(
    value=team,
    format='png',
    width=450,
    height=368
)

In [38]:
from ipywidgets import TwoByTwoLayout

tab1 = TwoByTwoLayout(top_left=btn_folder,
                     bottom_left=btn_collection,
                     top_right = btn_rename_img,
                     bottom_right=btn_split_data,
                     align_items="center",
                     height="200px")

tab2 = TwoByTwoLayout(top_left=btn_observation,
                     bottom_left=btn_distribution,
                     top_right = btn_visualization,
                     align_items="center",
                     height="200px")

tab3 = VBox(children=[HBox(children=[VBox(children=[
                                     btn_acc_loss, accLabel,
                                    acc1
                                    ]),
                                     VBox(children=[
                                     btn_confusion_matrix, confLabel,
                                    conf1
                                    ]),
                                     VBox(children=[
                                     btn_epoch_cf,
                                    confB1, confELabel
                                    ]),
                                     VBox(children=[
                                     btn_roc,rocLabel,
                                    roc1
                                    ]),
                                     VBox(children=[
                                     btn_mis_images, misLabel,
                                    missed1
                                    ]),
                                     VBox(children=[
                                     btn_hidden_layers, hiddenLabel,
                                    hlayer1
                                    ])
                    ]),
                      HBox(children=[VBox(children=[
                                     btn_layer_info, layerLabel,
                                    layer1
                                    ]),
                                     VBox(children=[
                                     btn_activations, actLabel,
                                    act1
                                    ]),
                                     VBox(children=[
                                     btn_heatmaps, heatLabel,
                                    heatmaps1
                                    ]),
                                     VBox(children=[
                                     btn_weights, weightLabel,
                                    weights1
                                    ]),
                                     VBox(children=[
                                     btn_inbound, inLabel,
                                    in1
                                    ]),
                                     VBox(children=[
                                     btn_outbound, outLabel,
                                    out1
                                    ])
                      ])
                ])

tab4 = VBox(children=[color_buttons,
                      HBox(children=[title_textbox,
                                     color_picker,
                                     grid_button]),
                                     ])

tab5 = HBox(children=[team1,
                      HBox(children=[title_textbox,
                                     color_picker,
                                     grid_button]),
                                     ])

In [39]:
import random
from IPython.display import clear_output
from ipywidgets import Output
from IPython.display import display

label = widgets.HTML("<h3 class = 'text-center', style = 'color:purple'> AI Data Management and Model Tracking")

tab = widgets.Tab(children=[tab1, tab2, tab3, tab4, tab5])
tab.set_title(0, 'Data Management')
tab.set_title(1, 'Data Analysis')
tab.set_title(2, 'Quality Tracking')
tab.set_title(3, 'AI Model Tracking')
tab.set_title(4, 'About')
VBox(children=[label,tab, button])

output = widgets.Output()
out = Output()
display(out)

#=============================================Button events for tab Data Management===========================================#
def on_build_dataset_clicked(b):
    with out:
        clear_output(True)
        ds.build_dataset()   
        
def on_collect_data_clicked(b):
    with out:
        clear_output(True)
        ds.collect_images()
        
                
def on_rename_img_clicked(b):
    with out:
        clear_output(True)
        ds.rename_img()   
        
def on_split_data_clicked(b):
    with out:
        clear_output(True)
        for class_name in ['phylloxera', 'mealybug', 'black_measles', 'black_rot', 'leaf_blight', 'healthy']:
            dn.train_test_split_data(ds.data, class_name)

#=============================================Button events for tab Data Analysis=============================================#
            
def on_observe_data_clicked(a):
    with out:
        clear_output(True)
        do.show_sample(ds.data)   
        
def on_data_distribution_clicked(d):
    with out:
        clear_output(True)
        ds.data_dist()
        
                
def on_data_visualization_clicked(e):
    with out:
        clear_output(True)
        dv.plot_scatter(nomalized_data_tsne, labels);  
        
#=============================================Button events for tab Quality Tracking==========================================#
            
def on_accloss_clicked(a):
    with out:
        clear_output(True)
       
        import cv2
        import glob
        import matplotlib.pyplot as plt
        import PIL

        images = [cv2.imread(file) for file in glob.glob('output/*.PNG')]
        for im in images: 
            plt.figure(figsize=(8,7))
            plt.imshow(im)
            plt.show()
        
def on_con_mat_clicked(d):
    with out:
        clear_output(True)
        from libraries import create_diagram as cd
        dependencies = {
                       'precision': precision,
                       'recall': recall, 
                       'f1_score': f1_score}

        model = load_model('diseasepest-scratch.h5', custom_objects=dependencies)

        # Display confusion matrix and classification report
        cd.collect_metrics(model, generator=valid_test_gen, steps=total_valid)     
        
def on_conf_clicked(d):
    with out:
        clear_output(True)
        import cv2
        import glob
        import matplotlib.pyplot as plt
        import PIL

        images = [cv2.imread(file) for file in glob.glob('performance_vizualizations/confusion_matrix*.PNG')]
        for im in images: 
            plt.figure(figsize=(20,19))
            plt.imshow(im)
            plt.show() 
            
def on_roc_clicked(d):
    with out:
        clear_output(True)
        import cv2
        import glob
        import matplotlib.pyplot as plt
        import PIL

        images = [cv2.imread(file) for file in glob.glob('performance_vizualizations/roc_curve*.PNG')]
        for im in images: 
            plt.figure(figsize=(20,19))
            plt.imshow(im)
            plt.show() 
                
def on_mis_images_clicked(e):
    with out:
        clear_output(True)
        from model_impl import get_misclassified_img as mi, display_mis_img as dmi
        #importlib.reload(mi)
        #importlib.reload(dmi)
        dependencies = {
                       'precision': precision,
                       'recall': recall, 
                       'f1_score': f1_score}

        model = load_model('diseasepest-scratch.h5', custom_objects=dependencies)
        paths = mi.get_misclassified_images(model, valid_test_gen, total_valid)

        paths
        dmi.display_misclassified_images(paths, rows = 19, columns = 4)
        
def on_hidden_layers_clicked(a):
    with out:
        clear_output(True)
        from model_impl import display_layer_info as li
        
        dependencies = {
                       'precision': precision,
                       'recall': recall, 
                       'f1_score': f1_score}

        model = load_model('diseasepest-scratch.h5', custom_objects=dependencies)
        li.print_hidden_layers(model)
                   
        
def on_layer_info_clicked(d):
    with out:
        clear_output(True)
        from model_impl import display_layer_info as li
        
        dependencies = {
                       'precision': precision,
                       'recall': recall, 
                       'f1_score': f1_score}

        model = load_model('diseasepest-scratch.h5', custom_objects=dependencies)
        li.print_layer_info(model)
        
def on_activations_clicked(e):
    with out:
        clear_output(True)
        from model_impl import display_activations as da

        dependencies = {
                       'precision': precision,
                       'recall': recall, 
                       'f1_score': f1_score}
        
        from libraries import dialog
        # get selected file's path
        path = dialog.gui_fname()
        path = path.decode("utf-8")

        # load the saved model
        model = load_model('diseasepest-scratch.h5', custom_objects=dependencies)
        
        # display activations
        da.display_activations1(path= path, input_mod=model)
        
def on_heatmaps_clicked(a):
    with out:
        clear_output(True)
        from model_impl import display_activations as da
        dependencies = {
                       'precision': precision,
                       'recall': recall, 
                       'f1_score': f1_score}
        
        from libraries import dialog
        # get selected file's path
        path = dialog.gui_fname()
        path = path.decode("utf-8")

        # load the saved model
        model = load_model('diseasepest-scratch.h5', custom_objects=dependencies)
        
        # display heatmaps
        da.display_heatmAps(path = path, input_mod = model)

def on_weights_clicked(d):
    with out:
        clear_output(True)
        from model_impl import weights_bias as wb
        dependencies = {
                       'precision': precision,
                       'recall': recall, 
                       'f1_score': f1_score}

        model = load_model('diseasepest-scratch.h5', custom_objects=dependencies)
        wb.extract_weights_bias(model)
   
                
def on_inbound_clicked(e):
    with out:
        clear_output(True)
        from model_impl import inbound_outbound_nodes as ion
        dependencies = {
                       'precision': precision,
                       'recall': recall, 
                       'f1_score': f1_score}

        model = load_model('diseasepest-scratch.h5', custom_objects=dependencies)
        ion.get_inbound_nodes(model)

        
def on_outbound_clicked(e):
    with out:
        clear_output(True)
        from model_impl import inbound_outbound_nodes as ion
        dependencies = {
                       'precision': precision,
                       'recall': recall, 
                       'f1_score': f1_score}

        model = load_model('diseasepest-scratch.h5', custom_objects=dependencies)
        ion.get_outbound_nodes(model)


#=============================================Button implementation for tab Data Management===================================#        
btn_folder.on_click(on_build_dataset_clicked)
btn_collection.on_click(on_collect_data_clicked)
btn_rename_img.on_click(on_rename_img_clicked)
btn_split_data.on_click(on_split_data_clicked)

#=============================================Button implementation for tab Data Analysis=====================================#
btn_observation.on_click(on_observe_data_clicked)
btn_distribution.on_click(on_data_distribution_clicked)
btn_visualization.on_click(on_data_visualization_clicked)

#=============================================Button implementation for tab Quality Tracking=================================#
btn_acc_loss.on_click(on_accloss_clicked)
btn_confusion_matrix.on_click(on_con_mat_clicked)
btn_epoch_cf.on_click(on_conf_clicked)
btn_roc.on_click(on_roc_clicked)
btn_mis_images.on_click(on_mis_images_clicked)
btn_hidden_layers.on_click(on_hidden_layers_clicked)
btn_layer_info.on_click(on_layer_info_clicked)
btn_activations.on_click(on_activations_clicked)
btn_heatmaps.on_click(on_heatmaps_clicked)
btn_weights.on_click(on_weights_clicked)
btn_inbound.on_click(on_inbound_clicked)
btn_outbound.on_click(on_outbound_clicked)

Output()